Les données de temperature sont pris de https://climat.meteo.gc.ca/index_f.html

In [1]:
import json
import pandas as pd
from scipy import interpolate
import numpy as np

In [2]:
# test = [{"time":i, "value":i*2} for i in range(10)]
# # tmp = json.dumps(test) #convert to json

# labels = [i for i in range(100)]
# temp_values = [i*2 for i in range(100)]
# hum_values = [20-i*0.3 for i in range(100)]
# water_values = [60-i*0.5 for i in range(100)]
# pH_values = [6-0.005*i for i in range(100)]
# CO2_values = [800-0.04*i for i in range(100)]
# Sun_values = [600+10*i for i in range(100)]

# test = {"labels":labels,"temp_values":temp_values,"hum_values":hum_values,"water_values":water_values,"pH_values":pH_values,"CO2_values":CO2_values,"Sun_values":Sun_values}

# print(test)
# # print(tmp)
# with open('data.json', 'w') as fp:
#     json.dump(test, fp)

In [3]:
temps_data = pd.read_csv("fr_climat_horaires_QC_702S006_07-2022_P1H.csv",decimal=',')
temps_data = temps_data[["Jour", "Heure (HNL)","Temp (°C)","Hum. rel (%)","Hauteur de précip. (mm)"]]
temps_data["Heure (HNL)"] = (temps_data["Heure (HNL)"].str.replace(":00","")).astype(int)
temps_data = temps_data[(temps_data["Jour"]>=7) & (temps_data["Jour"]<=13)]
temps_data

,Jour,Heure (HNL),Temp (°C),Hum. rel (%),Hauteur de précip. (mm)
144,7,0,17.5,60,0.0
145,7,1,16.8,63,0.2
146,7,2,15.7,66,0.0
147,7,3,15.2,68,0.0
148,7,4,15.1,64,0.0
...,...,...,...,...,...
307,13,19,23.1,41,0.0
308,13,20,22.6,43,0.0
309,13,21,20.5,54,0.0
310,13,22,19.3,61,0.0


In [4]:
# print(list(temps_data["Hum. rel (%)"]))

# day_temp_data = temps_data.groupby("Jour").mean(numeric_only=True)
# # day_temp_data

In [6]:
real_time = []
temp_values = [[],[],[]] 
hum_values = [[],[],[]]
water_values = [[],[],[]]
pH_values = [[],[],[]]
co2_values = []
sun_values = []
times = []

time_len = [1000,24*5,7*24]
times.append(list(np.linspace(0, time_len[0], num=time_len[0])))
times.append(list(np.linspace(0, 24, num=time_len[1])))
times.append(list(np.linspace(0, 7, num=time_len[2])))

for j in time_len:
    times.append([i for i in range(j)])
    co2_values.append([800-0.04*i for i in range(j)])
    sun_values.append([600+10*i for i in range(j)])


rng = np.random.default_rng(seed=42)

# for week
ratio2 = 2.5
f1 = interpolate.interp1d(times[2], temps_data["Hum. rel (%)"], kind='cubic')
f2 = interpolate.interp1d(times[2], temps_data["Temp (°C)"], kind='cubic')
mod_time_2 = list(np.linspace(0, 7, num=int(time_len[2]*ratio2)))

hum_tmp = f1(mod_time_2)
hum_values[2] = list(hum_tmp)

temp_tmp = f2(mod_time_2)
temp_values[2] = list(temp_tmp)

w_noise = rng.integers(low=-4, high=5, size=int(time_len[2]*ratio2))
w_noise[w_noise < 0] = 0
water_tmp = np.cumsum((100-hum_tmp)*w_noise*np.exp((1+temp_tmp-np.mean(temp_tmp))/2))/4
water_values[2] = list(water_tmp)

ph_noise = rng.integers(low=-1, high=2, size=int(time_len[2]*ratio2))
ph_noise[ph_noise < 0] = 0

pH_tmp = 6.6 -np.cumsum(np.log(1+water_tmp*100)*0.00008) + np.cumsum(ph_noise*0.003)
# pH_tmp = 6.6 + np.cumsum(-np.log(1+water_tmp*2)*0.000065 - water_tmp*0.000001 + ph_noise*np.log(1+water_tmp)*0.003)
pH_tmp = np.mean(pH_tmp)+((pH_tmp-np.mean(pH_tmp))/10)
pH_values[2] = list(pH_tmp)

# for day
ratio1 = 2.75
f1 = interpolate.interp1d(times[2][-25:], temps_data["Hum. rel (%)"][-49:-24], kind='cubic')
f2 = interpolate.interp1d(times[2][-25:], temps_data["Temp (°C)"][-49:-24], kind='cubic')
tmp_x = np.linspace(6,7, num=len(water_values[2][-int(49*ratio2):-int(18*ratio2)]))
f3 = interpolate.interp1d(tmp_x, water_values[2][-int(49*ratio2):-int(18*ratio2)], kind='linear')
# print(times[2][-25:],new_x)
new_x = list(np.linspace(6,7, num=int(time_len[1]*ratio1)))
times[1] = list(np.linspace(0, 24, num=int(time_len[1]*ratio1)))
hum_values[1] = list(f1(new_x))
temp_values[1] = list(f2(new_x))

water_tmp = f3(new_x)
water_values[1] = list(water_tmp-np.min(water_tmp))

ph_noise = rng.integers(low=-1, high=2, size=int(time_len[1]*ratio1))
ph_noise[ph_noise < 0] = 0
pH_tmp = 6.6 -np.cumsum(np.log(1+water_tmp*100)*0.00008) + np.cumsum(ph_noise*0.0035)
pH_tmp = np.mean(pH_tmp)+((pH_tmp-np.mean(pH_tmp))/10)
pH_values[1] = list(pH_tmp)

### for real time
tmp = list(np.linspace(0,time_len[0], num=int(time_len[1]*ratio1)))
f1 = interpolate.interp1d(tmp, hum_values[1], kind='linear')
f2 = interpolate.interp1d(tmp, temp_values[1], kind='linear')
f3 = interpolate.interp1d(tmp, water_values[1], kind='linear')

# new_x = list(np.linspace(0,time_len[0], num=int(time_len[0])))
times[0] = list(np.linspace(0,time_len[0], num=int(time_len[0])))
noise = rng.integers(low=1, high=50, size=time_len[0])

hum_tmp = f1(times[0])
hum_tmp = np.mean(hum_tmp)+((hum_tmp-np.mean(hum_tmp))/10)
hum_tmp = hum_tmp+(noise/100)
hum_values[0] = list(hum_tmp)

temp_tmp = f2(times[0])
temp_tmp = np.mean(temp_tmp)+((temp_tmp-np.mean(temp_tmp))/5)
temp_tmp = temp_tmp+(noise/1000)
temp_values[0] = list(temp_tmp)

water_tmp = f3(times[0])
water_tmp = (water_tmp - np.min(water_tmp))/2 +(noise/10)
water_values[0] = list(water_tmp)

ph_noise = rng.integers(low=-1, high=2, size=time_len[0])
ph_noise[ph_noise < 0] = 0
pH_tmp = 6.6 -np.cumsum(np.log(1+water_tmp*100)*0.00008) + np.cumsum(ph_noise*0.0023)
pH_tmp = np.mean(pH_tmp)+((pH_tmp-np.mean(pH_tmp))/10)
pH_values[0] = list(pH_tmp)

times[2] = mod_time_2

test = {"times":times,"temp_values":temp_values,"hum_values":hum_values,"water_values":water_values,"pH_values":pH_values,"CO2_values":co2_values,"Sun_values":sun_values}
# print(test)
# print(tmp)
with open('data.json', 'w') as fp:
    json.dump(test, fp)